In [1]:
!pip install paho-mqtt pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.6 MB/s eta 0:00:00


In [ ]:
# Importação das bibliotecas necessárias
import paho.mqtt.client as mqtt
import json
import time
import threading
import pandas as pd
from datetime import datetime, timezone, timedelta # Importações para fuso horário
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output

# =========== Configurações do MQTT =============
# Devem ser EXATAMENTE as mesmas do código que está no seu ESP32 com o sensor MQ-2
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
MQTT_TOPIC = "br/com/meuprojeto/esp32c3/mq2" # << TÓPICO ATUALIZADO para o sensor de gás

# =========== Configurações de Plotagem =============
READ_INTERVAL = 5  # Intervalo de atualização do gráfico (em segundos)
MAX_POINTS = 100   # Quantidade máxima de pontos a manter no gráfico

# --- Fuso Horário de Brasília (UTC-3) ---
# Define o fuso horário para corrigir a hora mostrada no gráfico
br_timezone = timezone(timedelta(hours=-3))

# --- Variáveis globais para buffer de dados ---
# Dicionário para armazenar os dados do sensor MQ-2 em memória
data_buffer = {
    "timestamp": [],
    "valorAnalogico": [],
    "statusDigital": [] # Usaremos 1 para GÁS_DETECTADO e 0 para NORMAL
}
# O 'lock' é um mecanismo de segurança para evitar que a thread de rede (que recebe os dados)
# e a thread principal (que plota os dados) tentem modificar o buffer ao mesmo tempo.
buffer_lock = threading.Lock()

# --- Funções MQTT ---

# Esta é uma função "callback". Ela será chamada automaticamente pela biblioteca MQTT
# toda vez que uma nova mensagem chegar no tópico em que nos inscrevemos.
def on_message(client, userdata, msg):
    try:
        # A mensagem chega como um objeto de bytes, então decodificamos para uma string
        payload_str = msg.payload.decode()
        print(f"Mensagem recebida: {payload_str}")

        # Converte a string JSON para um dicionário Python para podermos usar os dados
        data = json.loads(payload_str)

        # Pega os valores de 'valorAnalogico' e 'status' do dicionário de forma segura
        analog_value = data.get("valorAnalogico")
        status_str = data.get("status")

        # Só continua se os dados existirem no JSON
        if analog_value is not None and status_str is not None:
            # Converte o status de texto para um valor numérico para o gráfico (0 ou 1)
            digital_status = 1 if status_str == "GAS_DETECTADO" else 0

            # Usa o 'lock' para garantir que a escrita no buffer seja segura
            with buffer_lock:
                # Pega a hora atual em UTC e converte para o fuso de Brasília (BRT)
                timestamp_brt = datetime.now(timezone.utc).astimezone(br_timezone)
                data_buffer["timestamp"].append(timestamp_brt)

                data_buffer["valorAnalogico"].append(analog_value)
                data_buffer["statusDigital"].append(digital_status)

                # Mantém o buffer com no máximo MAX_POINTS, removendo o mais antigo se necessário
                if len(data_buffer["timestamp"]) > MAX_POINTS:
                    for key in data_buffer:
                        data_buffer[key].pop(0)

    except Exception as e:
        print(f"Erro ao processar mensagem: {e}")

# Função para plotar os dados que estão no buffer
def plot_buffer():
    # Usa o 'lock' para fazer uma cópia segura dos dados para a plotagem
    with buffer_lock:
        df = pd.DataFrame(data_buffer.copy())

    # Se o DataFrame estiver vazio, apenas exibe uma mensagem de espera
    if df.empty:
        print("Aguardando os primeiros dados do sensor MQ-2...")
        return

    # Cria a figura do gráfico com um eixo Y secundário para o status digital
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Adiciona a linha da Leitura Analógica (nível de gás)
    fig.add_trace(
        go.Scatter(x=df["timestamp"], y=df["valorAnalogico"], mode="lines+markers", name="Leitura Analógica (Gás)"),
        secondary_y=False
    )
    # Adiciona a linha do Status Digital (se há detecção ou não)
    fig.add_trace(
        go.Scatter(x=df["timestamp"], y=df["statusDigital"], mode="lines+markers", name="Detecção de Gás", line=dict(shape='hv')),
        secondary_y=True
    )

    # Configurações de layout e títulos do gráfico
    fig.update_layout(
        title_text="Monitoramento de Gás (MQ-2) em Tempo Real via MQTT",
        xaxis_title="Horário (Fuso de Brasília)"
    )
    fig.update_yaxes(title_text="Valor Analógico (Concentração)", secondary_y=False)
    fig.update_yaxes(
        title_text="Status da Detecção",
        secondary_y=True,
        tickvals=[0, 1],  # Define as posições dos marcadores no eixo
        ticktext=["Normal", "Gás Detectado"]  # Define os textos para os marcadores
    )

    # Limpa o output da célula e exibe o novo gráfico atualizado
    clear_output(wait=True)
    display(fig)

# --- Programa Principal ---
# O código dentro deste 'if' só roda quando executamos o script diretamente
if __name__ == "__main__":
    # 1. Cria o objeto do cliente MQTT
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
    # 2. Associa nossa função 'on_message' ao evento de recebimento de mensagens
    client.on_message = on_message

    # 3. Conecta ao Broker MQTT na internet
    print(f"Conectando ao Broker MQTT em {MQTT_BROKER}...")
    client.connect(MQTT_BROKER, MQTT_PORT, 60)

    # 4. Inscreve-se no tópico para começar a receber as mensagens
    print(f"Inscrevendo-se no tópico: {MQTT_TOPIC}")
    client.subscribe(MQTT_TOPIC)

    # 5. Inicia o loop de rede em uma thread separada.
    #    Isso é crucial: ele cuida da comunicação MQTT em segundo plano,
    #    liberando nosso programa principal para continuar e plotar o gráfico.
    client.loop_start()

    # 6. Loop principal para plotar o gráfico continuamente
    try:
        while True:
            plot_buffer()
            time.sleep(READ_INTERVAL)
    # Permite parar o programa de forma limpa com Ctrl+C no teclado
    except KeyboardInterrupt:
        print("Parando o cliente MQTT...")
        client.loop_stop() # Para a thread de rede do MQTT
        print("Programa finalizado.")

Mensagem recebida: {"valorAnalogico":2106,"status":"NORMAL"}
